In [690]:
import pandas as pd
import numpy as np

In [691]:
sff_preds_0 = pd.read_csv('/Users/santiagoromano/Documents/code/MasterThesis/results/inference/predictions_cluster_0_sff.csv')
sff_preds_1 = pd.read_csv('/Users/santiagoromano/Documents/code/MasterThesis/results/inference/predictions_cluster_1_sff.csv')
sff_preds_2 = pd.read_csv('/Users/santiagoromano/Documents/code/MasterThesis/results/inference/predictions_cluster_2_sff.csv')

sff_preds_0.drop(columns=['cant_vta_pred_sff_median'], inplace=True)
sff_preds_1.drop(columns=['cant_vta_pred_sff_median'], inplace=True)
sff_preds_2.drop(columns=['cant_vta_pred_sff_median'], inplace=True)

sff_preds = pd.concat([sff_preds_0, sff_preds_1, sff_preds_2], ignore_index=True)


mean_preds_0 = pd.read_csv('/Users/santiagoromano/Documents/code/MasterThesis/results/inference/predictions_cluster_0_mean_sale.csv')
mean_preds_1 = pd.read_csv('/Users/santiagoromano/Documents/code/MasterThesis/results/inference/predictions_cluster_1_mean_sale.csv')
mean_preds_2 = pd.read_csv('/Users/santiagoromano/Documents/code/MasterThesis/results/inference/predictions_cluster_2_mean_sale.csv')

mean_preds = pd.concat([mean_preds_0, mean_preds_1, mean_preds_2], ignore_index=True)

In [692]:
print(sff_preds.shape
      , mean_preds.shape)

sff_preds.columns

(19200, 6) (19200, 6)


Index(['pdv_codigo', 'fecha_comercial', 'codigo_barras_sku', 'cant_vta',
       'cluster_sku', 'cant_vta_pred_sff_mean'],
      dtype='object')

In [693]:
preds = sff_preds.merge(mean_preds, on=['pdv_codigo', 'fecha_comercial', 'codigo_barras_sku', 'cant_vta',
       'cluster_sku'], how='inner' )
preds

,pdv_codigo,fecha_comercial,codigo_barras_sku,cant_vta,cluster_sku,cant_vta_pred_sff_mean,cant_vta_pred_mean_pdv_sku
0,1,2024-11-01,7891991000826,2800.0,0,2583.40400,1925.000000
1,1,2024-11-02,7891991000826,5600.0,0,2319.03170,1925.000000
2,1,2024-11-03,7891991000826,350.0,0,1840.06240,1925.000000
3,1,2024-11-04,7891991000826,2450.0,0,1003.35065,1925.000000
4,1,2024-11-05,7891991000826,1050.0,0,1508.62660,1925.000000
...,...,...,...,...,...,...,...
19195,30,2024-11-26,7897395099329,7000.0,2,12934.63400,23846.666667
19196,30,2024-11-27,7897395099329,81550.0,2,12946.59300,23846.666667
19197,30,2024-11-28,7897395099329,23800.0,2,21864.17600,23846.666667
19198,30,2024-11-29,7897395099329,101850.0,2,42020.43000,23846.666667


In [694]:
def inference(start_simulation_date, end_simulation_date, preds, unit_cost , price_cost, cluster = None):
    preds = preds[(preds['fecha_comercial'] <= end_simulation_date) &
              (preds['fecha_comercial'] >= start_simulation_date)]
    
    if cluster is not None:
        preds = preds[preds['cluster_sku'] == cluster]
    
    group_sum = preds.groupby(['pdv_codigo', 'codigo_barras_sku'])[
    ['cant_vta_pred_sff_mean', 'cant_vta_pred_mean_pdv_sku']
    ].sum().reset_index()

    preds_merged = preds.merge(group_sum, on=['pdv_codigo', 'codigo_barras_sku'], suffixes=('', '_total'))

    preds_merged['sff_balance'] = preds_merged['cant_vta_pred_sff_mean_total'] - preds_merged.groupby(['pdv_codigo', 'codigo_barras_sku'])['cant_vta'].cumsum()
    preds_merged['mean_balance'] = preds_merged['cant_vta_pred_mean_pdv_sku_total'] - preds_merged.groupby(['pdv_codigo', 'codigo_barras_sku'])['cant_vta'].cumsum()

    final_balance = preds_merged[preds_merged['fecha_comercial'] == end_simulation_date][
    ['pdv_codigo', 'codigo_barras_sku', 'sff_balance', 'mean_balance']
    ]
    final_balance['sff_balance'] = round(final_balance['sff_balance'], 0)
    final_balance['mean_balance'] = round(final_balance['mean_balance'], 0)

    final_balance = final_balance.sort_values(by=['codigo_barras_sku', 'pdv_codigo']).reset_index(drop=True)

    final_balance['sff_cost'] = final_balance['sff_balance'].apply(
        lambda x: x * unit_cost if x > 0 else (x * price_cost)* -1
    )
    final_balance['mean_cost'] = final_balance['mean_balance'].apply(
        lambda x: x * unit_cost if x > 0 else (x * price_cost)* -1
    )
    final_balance['sff_stockout'] = final_balance['sff_balance'].apply(lambda x: True if x < 0 else False)
    final_balance['mean_stockout'] = final_balance['mean_balance'].apply(lambda x: True if x < 0 else False)
    final_balance['sff_stockout'].sum(), final_balance['mean_stockout'].sum()
    
    return final_balance


In [695]:
print(preds[['codigo_barras_sku', 'pdv_codigo']].drop_duplicates().shape)


print(preds[preds['cluster_sku'] == 0][['codigo_barras_sku', 'pdv_codigo']].drop_duplicates().shape)
print(preds[preds['cluster_sku'] == 1][['codigo_barras_sku', 'pdv_codigo']].drop_duplicates().shape)
print(preds[preds['cluster_sku'] == 2][['codigo_barras_sku', 'pdv_codigo']].drop_duplicates().shape)

(640, 2)
(219, 2)
(198, 2)
(223, 2)


In [696]:
start_simulation_date = '2024-11-01'
end_simulation_date = '2024-11-10'

unit_cost = 0.2
price_cost = 0.65
final_balance = inference(start_simulation_date, end_simulation_date, preds, unit_cost, price_cost)

benefit = final_balance['mean_cost'].sum() - final_balance['sff_cost'].sum()

print(final_balance['sff_cost'].sum(), final_balance['mean_cost'].sum())
print(benefit)

69664130.45000002 73317674.8
3653544.349999979


In [697]:
final_balance['sff_stockout'].value_counts(), final_balance['mean_stockout'].value_counts()

(sff_stockout
 True     377
 False    263
 Name: count, dtype: int64,
 mean_stockout
 True     439
 False    201
 Name: count, dtype: int64)

In [698]:
print(preds['cant_vta'].mean())

56952.178125


In [699]:
final_balance['sff_stockout'].value_counts(), final_balance['mean_stockout'].value_counts()

(sff_stockout
 True     377
 False    263
 Name: count, dtype: int64,
 mean_stockout
 True     439
 False    201
 Name: count, dtype: int64)

In [700]:
print('units')
print( 'overstock ssf ',final_balance[final_balance['sff_stockout'] == False][['sff_balance']].sum())
print(  'stockout ssf ', final_balance[final_balance['sff_stockout'] == True][['sff_balance']].sum())
print('-------------')
print( 'overstock mean ', final_balance[final_balance['mean_stockout'] == False][['mean_balance']].sum())
print(  'stockout mean ', final_balance[final_balance['mean_stockout'] == True][['mean_balance']].sum())
print('-------------')
print('AVG. SFF BALANCE', final_balance[['sff_balance']].mean())
print('AVG. MEAN BALANCE', final_balance[['mean_balance']].mean())
print('----------------------------------------')
print('costs')
print( 'overstock ssf ',final_balance[final_balance['sff_stockout'] == False][['sff_cost']].sum())
print(  'stockout ssff ', final_balance[final_balance['sff_stockout'] == True][['sff_cost']].sum())
print('-------------')
print( 'overstock mean  ', final_balance[final_balance['mean_stockout'] == False][['mean_cost']].sum())
print(  'stockout mean  ', final_balance[final_balance['mean_stockout'] == True][['mean_cost']].sum())

units
overstock ssf  sff_balance    41300546.0
dtype: float64
stockout ssf  sff_balance   -94467725.0
dtype: float64
-------------
overstock mean  mean_balance    36610765.0
dtype: float64
stockout mean  mean_balance   -101531572.0
dtype: float64
-------------
AVG. SFF BALANCE sff_balance   -83073.717188
dtype: float64
AVG. MEAN BALANCE mean_balance   -101438.760938
dtype: float64
----------------------------------------
costs
overstock ssf  sff_cost    8260109.2
dtype: float64
stockout ssff  sff_cost    61404021.25
dtype: float64
-------------
overstock mean   mean_cost    7322153.0
dtype: float64
stockout mean   mean_cost    65995521.8
dtype: float64


In [615]:
benefit / 640

29447.89140625

In [616]:
print(preds['cant_vta'].mean())

print(preds[preds['cluster_sku'] == 0]['cant_vta'].mean())

print(preds[preds['cluster_sku'] == 1]['cant_vta'].mean())

print(preds[preds['cluster_sku'] == 2]['cant_vta'].mean())



56952.178125
15345.791476407914
121911.59090909091
40135.29446935725


# Cluster 0

In [701]:
cluster = 0
final_balance = inference(start_simulation_date, end_simulation_date, preds, unit_cost, price_cost, cluster)

print(final_balance['sff_cost'].sum(), final_balance['mean_cost'].sum())
print(final_balance['mean_cost'].sum() - final_balance['sff_cost'].sum())

4782755.899999999 4289219.15
-493536.74999999907


In [702]:
final_balance['sff_stockout'].value_counts(), final_balance['mean_stockout'].value_counts()

(sff_stockout
 True     139
 False     80
 Name: count, dtype: int64,
 mean_stockout
 True     120
 False     99
 Name: count, dtype: int64)

In [703]:
print('units')
print( 'overstock ssf ',final_balance[final_balance['sff_stockout'] == False][['sff_balance']].sum())
print(  'stockout ssf ', final_balance[final_balance['sff_stockout'] == True][['sff_balance']].sum())
print('-------------')
print( 'overstock mean ', final_balance[final_balance['mean_stockout'] == False][['mean_balance']].sum())
print(  'stockout mean ', final_balance[final_balance['mean_stockout'] == True][['mean_balance']].sum())
print('-------------')
print('AVG. SFF BALANCE', final_balance[['sff_balance']].mean())
print('AVG. MEAN BALANCE', final_balance[['mean_balance']].mean())

print('----------------------------------------')
print('costs')
print( 'overstock ssf ',final_balance[final_balance['sff_stockout'] == False][['sff_cost']].sum())
print(  'stockout ssff ', final_balance[final_balance['sff_stockout'] == True][['sff_cost']].sum())
print('-------------')
print( 'overstock mean  ', final_balance[final_balance['mean_stockout'] == False][['mean_cost']].sum())
print(  'stockout mean  ', final_balance[final_balance['mean_stockout'] == True][['mean_cost']].sum())

units
overstock ssf  sff_balance    2351986.0
dtype: float64
stockout ssf  sff_balance   -6634398.0
dtype: float64
-------------
overstock mean  mean_balance    3782089.0
dtype: float64
stockout mean  mean_balance   -5435079.0
dtype: float64
-------------
AVG. SFF BALANCE sff_balance   -19554.392694
dtype: float64
AVG. MEAN BALANCE mean_balance   -7547.899543
dtype: float64
----------------------------------------
costs
overstock ssf  sff_cost    470397.2
dtype: float64
stockout ssff  sff_cost    4312358.7
dtype: float64
-------------
overstock mean   mean_cost    756417.8
dtype: float64
stockout mean   mean_cost    3532801.35
dtype: float64


# Cluster 1 

In [704]:
cluster = 1
final_balance = inference(start_simulation_date, end_simulation_date, preds, unit_cost, price_cost, cluster)

print(final_balance['sff_cost'].sum(), final_balance['mean_cost'].sum())
print(final_balance['mean_cost'].sum() - final_balance['sff_cost'].sum())

53741386.5 55376891.14999999
1635504.649999991


In [705]:
final_balance['sff_stockout'].value_counts(), final_balance['mean_stockout'].value_counts()

(sff_stockout
 True     110
 False     88
 Name: count, dtype: int64,
 mean_stockout
 True     150
 False     48
 Name: count, dtype: int64)

In [706]:
print('units')
print( 'overstock ssf ',final_balance[final_balance['sff_stockout'] == False][['sff_balance']].sum())
print(  'stockout ssf ', final_balance[final_balance['sff_stockout'] == True][['sff_balance']].sum())
print('-------------')
print( 'overstock mean ', final_balance[final_balance['mean_stockout'] == False][['mean_balance']].sum())
print(  'stockout mean ', final_balance[final_balance['mean_stockout'] == True][['mean_balance']].sum())
print('-------------')
print('AVG. SFF BALANCE', final_balance[['sff_balance']].mean())
print('AVG. MEAN BALANCE', final_balance[['mean_balance']].mean())
print('----------------------------------------')
print('costs')
print( 'overstock ssf ',final_balance[final_balance['sff_stockout'] == False][['sff_cost']].sum())
print(  'stockout ssff ', final_balance[final_balance['sff_stockout'] == True][['sff_cost']].sum())
print('-------------')
print( 'overstock mean  ', final_balance[final_balance['mean_stockout'] == False][['mean_cost']].sum())
print(  'stockout mean  ', final_balance[final_balance['mean_stockout'] == True][['mean_cost']].sum())

units
overstock ssf  sff_balance    31181495.0
dtype: float64
stockout ssf  sff_balance   -73084750.0
dtype: float64
-------------
overstock mean  mean_balance    28404136.0
dtype: float64
stockout mean  mean_balance   -76455483.0
dtype: float64
-------------
AVG. SFF BALANCE sff_balance   -211632.60101
dtype: float64
AVG. MEAN BALANCE mean_balance   -242683.570707
dtype: float64
----------------------------------------
costs
overstock ssf  sff_cost    6236299.0
dtype: float64
stockout ssff  sff_cost    47505087.5
dtype: float64
-------------
overstock mean   mean_cost    5680827.2
dtype: float64
stockout mean   mean_cost    49696063.95
dtype: float64


# Cluster 2

In [707]:
cluster = 2
final_balance = inference(start_simulation_date, end_simulation_date, preds, unit_cost, price_cost, cluster)

print(final_balance['sff_cost'].sum(), final_balance['mean_cost'].sum())
print(final_balance['mean_cost'].sum() - final_balance['sff_cost'].sum())

11139988.05 13651564.5
2511576.4499999993


In [708]:
final_balance['sff_stockout'].value_counts(), final_balance['mean_stockout'].value_counts()

(sff_stockout
 True     128
 False     95
 Name: count, dtype: int64,
 mean_stockout
 True     169
 False     54
 Name: count, dtype: int64)

In [709]:
print('units')
print( 'overstock ssf ',final_balance[final_balance['sff_stockout'] == False][['sff_balance']].sum())
print(  'stockout ssf ', final_balance[final_balance['sff_stockout'] == True][['sff_balance']].sum())
print('-------------')
print( 'overstock mean ', final_balance[final_balance['mean_stockout'] == False][['mean_balance']].sum())
print(  'stockout mean ', final_balance[final_balance['mean_stockout'] == True][['mean_balance']].sum())
print('-------------')
print('AVG. SFF BALANCE', final_balance[['sff_balance']].mean())
print('AVG. MEAN BALANCE', final_balance[['mean_balance']].mean())
print('----------------------------------------')
print('costs')
print( 'overstock ssf ',final_balance[final_balance['sff_stockout'] == False][['sff_cost']].sum())
print(  'stockout ssff ', final_balance[final_balance['sff_stockout'] == True][['sff_cost']].sum())
print('-------------')
print( 'overstock mean  ', final_balance[final_balance['mean_stockout'] == False][['mean_cost']].sum())
print(  'stockout mean  ', final_balance[final_balance['mean_stockout'] == True][['mean_cost']].sum())

units
overstock ssf  sff_balance    7767065.0
dtype: float64
stockout ssf  sff_balance   -14748577.0
dtype: float64
-------------
overstock mean  mean_balance    4424540.0
dtype: float64
stockout mean  mean_balance   -19641010.0
dtype: float64
-------------
AVG. SFF BALANCE sff_balance   -31307.2287
dtype: float64
AVG. MEAN BALANCE mean_balance   -68235.29148
dtype: float64
----------------------------------------
costs
overstock ssf  sff_cost    1553413.0
dtype: float64
stockout ssff  sff_cost    9586575.05
dtype: float64
-------------
overstock mean   mean_cost    884908.0
dtype: float64
stockout mean   mean_cost    12766656.5
dtype: float64


# QC

In [202]:
class Metrics:
    @staticmethod
    def mean_squared_error(y_true, y_pred):
        """
        Calculate Mean Squared Error (MSE).
        
        Parameters:
        y_true (array-like): Array of real values.
        y_pred (array-like): Array of predicted values.
        
        Returns:
        float: Mean Squared Error.
        """
        n = len(y_true)
        mse = np.sum((y_true - y_pred) ** 2) / n
        return mse

    @staticmethod
    def root_mean_squared_error(y_true, y_pred):
        """
        Calculate Root Mean Squared Error (RMSE).
        
        Parameters:
        y_true (array-like): Array of real values.
        y_pred (array-like): Array of predicted values.
        
        Returns:
        float: Root Mean Squared Error.
        """
        mse = Metrics.mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        return rmse
    
    def calculate_metrics(self, group):
        y_true = group['cant_vta']
        metrics = {}

        # Identify all columns that start with 'cant_vta_pred_'
        pred_columns = [col for col in group.columns if col.startswith('cant_vta_pred_')]

        for col in pred_columns:
            mse = round(Metrics.mean_squared_error(y_true, group[col]),1)
            rmse = round(Metrics.root_mean_squared_error(y_true, group[col]),1)
            metrics[f'mse_{col}'] = mse
            metrics[f'rmse_{col}'] = rmse

        return pd.Series(metrics)

    def create_summary_dataframe(self, test_df):
        # TODO: ANALIZE WHAT TO DO WHEN THERE ARE NULLS IN THE REAL VALUES
        summary_df = test_df.groupby(['pdv_codigo', 'codigo_barras_sku']).apply(self.calculate_metrics).reset_index()

        rmse_columns = [col for col in summary_df.columns if col.startswith('rmse_')]
        mse_columns = [col for col in summary_df.columns if col.startswith('mse_')]

        
        summary_df['best_rmse'] = summary_df[rmse_columns].idxmin(axis=1)
        summary_df['best_mse'] = summary_df[mse_columns].idxmin(axis=1)

        return summary_df

In [208]:
preds_0 = preds[preds['cluster_sku'] == 0]
preds_1 = preds[preds['cluster_sku'] == 1]
preds_2 = preds[preds['cluster_sku'] == 2]

In [204]:
summary_df = Metrics().create_summary_dataframe(preds)

/var/folders/g4/21kk_yqn4h75mgg_8hcjcsx80000gn/T/ipykernel_44093/3537863226.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary_df = test_df.groupby(['pdv_codigo', 'codigo_barras_sku']).apply(self.calculate_metrics).reset_index()


In [206]:
print(summary_df['rmse_cant_vta_pred_sff_mean'].median(), summary_df['rmse_cant_vta_pred_sff_mean'].mean())

print(summary_df['rmse_cant_vta_pred_mean_pdv_sku'].median(), summary_df['rmse_cant_vta_pred_mean_pdv_sku'].mean())

15715.3 54648.73187500001
17826.6 59399.79718749999


In [209]:
sff_0 = Metrics().create_summary_dataframe(preds_0)
sff_1 = Metrics().create_summary_dataframe(preds_1)
sff_2 = Metrics().create_summary_dataframe(preds_2)


/var/folders/g4/21kk_yqn4h75mgg_8hcjcsx80000gn/T/ipykernel_44093/3537863226.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary_df = test_df.groupby(['pdv_codigo', 'codigo_barras_sku']).apply(self.calculate_metrics).reset_index()
/var/folders/g4/21kk_yqn4h75mgg_8hcjcsx80000gn/T/ipykernel_44093/3537863226.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary_df = test_df.groupby(['pdv_codigo'

In [211]:
print(sff_0['rmse_cant_vta_pred_sff_mean'].median(),sff_0['rmse_cant_vta_pred_sff_mean'].mean())
print(sff_0['rmse_cant_vta_pred_mean_pdv_sku'].median(),sff_0['rmse_cant_vta_pred_mean_pdv_sku'].mean())

print('-------------------------')

print(sff_1['rmse_cant_vta_pred_sff_mean'].median(),sff_1['rmse_cant_vta_pred_sff_mean'].mean())
print(sff_1['rmse_cant_vta_pred_mean_pdv_sku'].median(),sff_1['rmse_cant_vta_pred_mean_pdv_sku'].mean())

print('-------------------------')

print(sff_2['rmse_cant_vta_pred_sff_mean'].median(),sff_2['rmse_cant_vta_pred_sff_mean'].mean())
print(sff_2['rmse_cant_vta_pred_mean_pdv_sku'].median(),sff_2['rmse_cant_vta_pred_mean_pdv_sku'].mean())

6252.3 9028.149315068495
6626.7 9245.24794520548
-------------------------
29804.050000000003 135440.87323232324
38956.85 147077.27979797978
-------------------------
16190.5 27716.28161434978
19162.7 30806.54484304933
